In [1]:
import Pkg
Pkg.update()

pkgs = [
    "Revise",
    "MetaGraphs",
    "Graphs",
    "JSON",
    "uCSV",
    "DataFrames",
    "Dates",
    "Primes",
    "Kmers",
    "BioSequences",
    "FASTX",
    "ProgressMeter",
    "Random",
    "StatsBase"
]

Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

# Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

    Updating registry at `/opt/julia/registries/General`
  No Changes to `~/work/Mycelia/Project.toml`
  No Changes to `~/work/Mycelia/Manifest.toml`
Precompiling project...
  ✓ Plots
  ✓ StatsPlots
  ✓ Mycelia
  3 dependencies successfully precompiled in 77 seconds (236 already precompiled)
   Resolving package versions...
  No Changes to `~/work/Mycelia/Project.toml`
  No Changes to `~/work/Mycelia/Manifest.toml`


In [2]:
#papermill parameters
base_directory = "$(homedir())/workspace/sars-cov2-pangenome-analysis/"
sequences_directory = "$(base_directory)/data/sequences"
metadata_file = "$(base_directory)/metadata/sequences.csv"
working_directory = "$(homedir())/workspace/scratch/$(Dates.today())"

"/home/jovyan/workspace/scratch/2022-10-29"

In [3]:
neo4j_username = "neo4j"

# remote_neo4j_address = ENV["NEO4J_URL"]
# remote_neo4j_password = ENV["NEO4J_PASSWORD"]
local_neo4j_bolt_address = "bolt://localhost:7687"
local_neo4j_http_address = "neo4j://localhost:7474"
# local_neo4j_password = "neo4j"
# local_neo4j_password = Random.randstring(7)
local_neo4j_password = "ii0sRIc"

neo4j_local_import_directory = "/home/jovyan/.local/neo4j-community-4.4.11/import"

"/home/jovyan/.local/neo4j-community-4.4.11/import"

In [4]:
NEO4J_PATH="/home/jovyan/.local/neo4j-community-4.4.11"

"/home/jovyan/.local/neo4j-community-4.4.11"

In [5]:
run(`$(NEO4J_PATH)/bin/neo4j stop`)
run(`$(NEO4J_PATH)/bin/neo4j start`)
run(`$(NEO4J_PATH)/bin/neo4j status`)

Neo4j is not running.
Directories in use:
home:         /home/jovyan/.local/neo4j-community-4.4.11
config:       /home/jovyan/.local/neo4j-community-4.4.11/conf
logs:         /home/jovyan/.local/neo4j-community-4.4.11/logs
plugins:      /home/jovyan/.local/neo4j-community-4.4.11/plugins
import:       /home/jovyan/.local/neo4j-community-4.4.11/import
data:         /home/jovyan/.local/neo4j-community-4.4.11/data
certificates: /home/jovyan/.local/neo4j-community-4.4.11/certificates
licenses:     /home/jovyan/.local/neo4j-community-4.4.11/licenses
run:          /home/jovyan/.local/neo4j-community-4.4.11/run
Starting Neo4j.
Started neo4j (pid:67771). It is available at http://localhost:7474
There may be a short delay until the server is ready.
Neo4j is running at pid 67771


Process(`/home/jovyan/.local/neo4j-community-4.4.11/bin/neo4j status`, ProcessExited(0))

In [ ]:
# # # update password - only need to do this on initial setup
# cmd = "ALTER CURRENT USER SET PASSWORD FROM 'neo4j' TO '$(local_neo4j_password)'"
# run(Mycelia.cypher(cmd, address=local_neo4j_bolt_address, password="neo4j", username="neo4j", database="system"))

In [6]:
mkpath(working_directory)
cd(working_directory)

In [10]:
# f = metadata_file
n = 10^1+1 # complete
# n = 10^2+1
# n = 10^3+1
# n = 10^4+1
# n = 10^5+1
# n = 10^6+1
# n = countlines(metadata_file)
f = open(`head -n $(n) $metadata_file`)
@time sequence_metadata = DataFrames.DataFrame(uCSV.read(f, quotes='"', header=1, typedetectrows=100)...)

  0.000918 seconds (1.88 k allocations: 258.312 KiB)


Row,Accession,SRA_Accession,Submitters,Release_Date,Pangolin,PangoVersions,Random_Sampling,Isolate,Species,Genus,Family,Molecule_type,Length,Sequence_Type,Nuc_Completeness,Genotype,Segment,Publications,Geo_Location,Country,USA,Host,Isolation_Source,Collection_Date,BioSample,GenBank_Title
,String,String,String,String,String,String,String,String,String,String,String,String,Int64,String,String,String,String,String,String,String,String,String,String,String,String,String
1,NC_045512.2,,"Wu,F., Zhao,S., Yu,B., Chen,Y.M., Wang,W., Song,Z.G., Hu,Y., Tao,Z.W., Tian,J.H., Pei,Y.Y., Yuan,M.L., Zhang,Y.L., Dai,F.H., Liu,Y., Wang,Q.M., Zheng,J.J., Xu,L., Holmes,E.C., Zhang,Y.Z., Baranov,P.V., Henderson,C.M., Anderson,C.B., Gesteland,R.F., Atkins,J.F., Howard,M.T., Robertson,M.P., Igel,H., Baertsch,R., Haussler,D., Ares,M. Jr., Scott,W.G., Williams,G.D., Chang,R.Y., Brian,D.A., Chen,Y.-M., Song,Z.-G., Tao,Z.-W., Tian,J.-H., Pei,Y.-Y., Zhang,Y.-L., Dai,F.-H., Wang,Q.-M., Zheng,J.-J., Zhang,Y.-Z.",2020-01-13T00:00:00Z,B,4.1.2/1.12/v0.1.10/0.3.17,,Wuhan-Hu-1,Severe acute respiratory syndrome-related coronavirus,Betacoronavirus,Coronaviridae,ssRNA(+),29903,RefSeq,complete,,,4,China,China,,Homo sapiens,,2019-12,,"Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome"
2,OP164776.1,,"Afrad Moon,M.H., Qadri,F.",2022-08-07T00:00:00Z,,4.1.2/1.12/v0.1.10/0.3.17,,icddrb-TND-07-1525,Severe acute respiratory syndrome-related coronavirus,Betacoronavirus,Coronaviridae,ssRNA(+),29723,GenBank,partial,,,,Bangladesh,Bangladesh,,Homo sapiens,,2022-06-14,,"Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/BGD/icddrb-TND-07-1525/2022 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope protein (E), membrane glycoprotein (M), ORF6 protein (ORF6), and ORF7a protein (ORF7a) genes, complete cds; ORF7b (ORF7b) and ORF8 protein (ORF8) genes, partial cds; and nucleocapsid phosphoprotein (N) and ORF10 protein (ORF10) genes, complete cds"
3,OP164777.1,,"Afrad Moon,M.H., Qadri,F.",2022-08-07T00:00:00Z,,4.1.2/1.12/v0.1.10/0.3.17,,ideSHi-IR-2986,Severe acute respiratory syndrome-related coronavirus,Betacoronavirus,Coronaviridae,ssRNA(+),29723,GenBank,partial,,,,Bangladesh,Bangladesh,,Homo sapiens,,2022-06-13,,"Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/BGD/ideSHi-IR-2986/2022 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope protein (E), membrane glycoprotein (M), ORF6 protein (ORF6), and ORF7a protein (ORF7a) genes, complete cds; ORF7b (ORF7b) and ORF8 protein (ORF8) genes, partial cds; and nucleocapsid phosphoprotein (N) and ORF10 protein (ORF10) genes, complete cds"
4,OP164778.1,,"Afrad Moon,M.H., Qadri,F.",2022-08-07T00:00:00Z,,4.1.2/1.12/v0.1.10/0.3.17,,ideSHi-IR-2989,Severe acute respiratory syndrome-related coronavirus,Betacoronavirus,Coronaviridae,ssRNA(+),29723,GenBank,partial,,,,Bangladesh,Bangladesh,,Homo sapiens,,2022-06-13,,"Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/BGD/ideSHi-IR-2989/2022 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope protein (E), membrane glycoprotein (M), ORF6 protein (ORF6), and ORF7a protein (ORF7a) genes, complete cds; ORF7b (ORF7b) and ORF8 protein (ORF8) genes, partial cds; and nucleocapsid phosphoprotein (N) and ORF10 protein (ORF10) genes, complete cds"
5,OP164779.1,,"Afrad Moon,M.H., Qadri,F.",2022-08-07T00:00:00Z,,4.1.2/1.12/v0.1.10/0.3.17,,ideSHi-IR-2991,Severe acute respiratory syndrome-related coronavirus,Betacoronavirus,Coronaviridae,ssRNA(+),29723,GenBank,partial,,,,Bangladesh,Bangladesh,,Homo sapiens,,2022-06-14,,"Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/BGD/ideSHi-IR-2991/2022 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope pr

In [11]:
fastx_files = map(x -> "$(sequences_directory)/$(x).fna", sequence_metadata[!, "Accession"])

10-element Vector{String}:
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/NC_045512.2.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164776.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164777.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164778.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164779.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164780.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164781.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164782.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164783.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164784.1.fna"

In [12]:
graph = Mycelia.initialize_graph()
graph = Mycelia.add_fastx_to_graph!(graph, fastx_files)
graph = Mycelia.add_metadata_from_table!(graph, sequence_metadata, identifier_column = "Accession")

{10, 0} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [13]:
# max_k = Mycelia.assess_dnamer_saturation(fastx_files)
# max_k = 31
# min_k=17
# max_k=17
min_k=max_k=31
kmer_sizes = Primes.primes(min_k, max_k)

1-element Vector{Int64}:
 31

In [14]:
for kmer_size in kmer_sizes
    @show kmer_size
    @time Mycelia.add_fasta_record_kmers_to_graph!(graph, kmer_size)
end

kmer_size = 31
  6.483784 seconds (23.62 M allocations: 1.884 GiB, 19.10% gc time)


In [15]:
for kmer_size in kmer_sizes
    @show kmer_size
    @time Mycelia.graph_to_gfa(graph, kmer_size)
end

kmer_size = 31
  1.639356 seconds (3.32 M allocations: 188.443 MiB, 28.76% gc time, 24.27% compilation time)


In [16]:
for gfa in filter(x -> occursin(r"\.gfa$", x), readdir(working_directory, join=true))
    @show gfa
    for nodewidth in [100, 500, 1000]
        gfa_img = "$(gfa).$(nodewidth).jpg"
        if !isfile(gfa_img)
            run(`Bandage image $gfa $gfa_img --deppower 1 --depwidth 1 --nodewidth $(nodewidth)`)
        end
    end
end

gfa = "/home/jovyan/workspace/scratch/2022-10-29/31.gfa"


QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jovyan'
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jovyan'
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jovyan'


In [17]:
run(`$(NEO4J_PATH)/bin/neo4j stop`)
run(`$(NEO4J_PATH)/bin/neo4j start`)
sleep(3)
run(`$(NEO4J_PATH)/bin/neo4j status`)

Stopping Neo4j....... stopped.
Directories in use:
home:         /home/jovyan/.local/neo4j-community-4.4.11
config:       /home/jovyan/.local/neo4j-community-4.4.11/conf
logs:         /home/jovyan/.local/neo4j-community-4.4.11/logs
plugins:      /home/jovyan/.local/neo4j-community-4.4.11/plugins
import:       /home/jovyan/.local/neo4j-community-4.4.11/import
data:         /home/jovyan/.local/neo4j-community-4.4.11/data
certificates: /home/jovyan/.local/neo4j-community-4.4.11/certificates
licenses:     /home/jovyan/.local/neo4j-community-4.4.11/licenses
run:          /home/jovyan/.local/neo4j-community-4.4.11/run
Starting Neo4j.
Started neo4j (pid:68097). It is available at http://localhost:7474
There may be a short delay until the server is ready.
Neo4j is running at pid 68097


Process(`/home/jovyan/.local/neo4j-community-4.4.11/bin/neo4j status`, ProcessExited(0))

In [ ]:
# Mycelia.list_databases(address=local_neo4j_bolt_address, password=local_neo4j_password)

In [20]:
Mycelia.create_node_constraints(graph, address=local_neo4j_bolt_address, password=local_neo4j_password)

cypher = `/home/jovyan/.local/cypher-shell/cypher-shell --address bolt://localhost:7687 --username neo4j --password ii0sRIc --format auto --database neo4j 'CREATE CONSTRAINT ON (t:\`FASTX.FASTA.Record\`) ASSERT t.identifier IS UNIQUE'`


An equivalent constraint already exists, 'Constraint( id=4, name='constraint_a0af4ea1', type='UNIQUENESS', schema=(:FASTX.FASTA.Record {identifier}), ownedIndex=3 )'.


cypher = `/home/jovyan/.local/cypher-shell/cypher-shell --address bolt://localhost:7687 --username neo4j --password ii0sRIc --format auto --database neo4j 'CREATE CONSTRAINT ON (t:\`Kmers.DNAKmer{31}\`) ASSERT t.identifier IS UNIQUE'`


An equivalent constraint already exists, 'Constraint( id=6, name='constraint_e07b72f7', type='UNIQUENESS', schema=(:Kmers.DNAKmer{31} {identifier}), ownedIndex=5 )'.


In [21]:
run(Mycelia.cypher("MATCH (n) DETACH DELETE n", address=local_neo4j_bolt_address, password=local_neo4j_password))
run(Mycelia.cypher("MATCH (n) RETURN count(n) as count", address=local_neo4j_bolt_address, password=local_neo4j_password))
# run(Mycelia.cypher("MATCH (n) DETACH DELETE n", address=local_neo4j_bolt_address, password=local_neo4j_password))

count
0


Process(`/home/jovyan/.local/cypher-shell/cypher-shell --address bolt://localhost:7687 --username neo4j --password ii0sRIc --format auto --database neo4j 'MATCH (n) RETURN count(n) as count'`, ProcessExited(0))

In [22]:
Mycelia.upload_nodes_to_neo4j(graph=graph, address=local_neo4j_bolt_address, password=local_neo4j_password, neo4j_import_directory=neo4j_local_import_directory)

┌ Info: uploading node_type => FASTX.FASTA.Record...
└ @ Mycelia /home/jovyan/work/Mycelia/src/database-interaction.jl:79


LoadError: UndefVarError: JSON not defined

In [23]:
run(Mycelia.cypher("MATCH (n) RETURN count(n) as count", address=local_neo4j_bolt_address, password=local_neo4j_password))

count
0


Process(`/home/jovyan/.local/cypher-shell/cypher-shell --address bolt://localhost:7687 --username neo4j --password ii0sRIc --format auto --database neo4j 'MATCH (n) RETURN count(n) as count'`, ProcessExited(0))

In [ ]:
Graphs.nv(graph)

In [ ]:
function upload_edges_to_neo4j(;graph, address, username="neo4j", password, format="auto", database="neo4j", neo4j_import_directory)
    
    edge_types = unique(MetaGraphs.props(graph, e)[:TYPE] for e in Graphs.edges(graph))
    for edge_type in edge_types
        @info "uploading edge_type => $(Mycelia.type_to_string(edge_type))..."
        edge_type_table = edge_type_to_dataframe(edge_type=edge_type, graph=graph)
        upload_edge_table(table=edge_type_table, address=address, password=password, neo4j_import_dir=neo4j_import_directory)
    end
    @info "done!"
end

function edge_type_to_dataframe(;edge_type, graph)
    edges = filter(e -> MetaGraphs.props(graph, e)[:TYPE] == edge_type, collect(Graphs.edges(graph)))
    edge_type_parameters = unique(reduce(vcat, map(e -> collect(keys(MetaGraphs.props(graph, e))), edges)))
    edge_type_table = DataFrames.DataFrame(Dict(p => [] for p in edge_type_parameters))
    edge_type_table[!, "src"] = String[]
    edge_type_table[!, "dst"] = String[]
    for edge in edges
        edge_dict = copy(MetaGraphs.props(graph, edge))
        edge_dict[:src] = string(MetaGraphs.props(graph, edge.src)[:identifier])
        edge_dict[:dst] = string(MetaGraphs.props(graph, edge.dst)[:identifier])
        push!(edge_type_table, edge_dict)
    end
    # normalize
    edge_type_table[!, "TYPE"] = Mycelia.type_to_string.(edge_type_table[!, "TYPE"])
    for column in names(edge_type_table)
        T = Union{unique(typeof.(edge_type_table[!, column]))...}
        if T <: AbstractDict
            edge_type_table[!, column] = map(d -> JSON.json(string(JSON.json(d))), edge_type_table[!, column])
        else
            edge_type_table[!, column] = JSON.json.(string.(edge_type_table[!, column]))
        end
    end
    return edge_type_table
end

function upload_edge_table(;table, window_size=1000, address, password, username="neo4j", database="neo4j", neo4j_import_dir)
    nrows = DataFrames.nrow(table)
    windows = (i:min(i+window_size-1,nrows) for i in 1:window_size:nrows)
    
    edge_types = unique(table[!, "TYPE"])
    @assert length(edge_types) == 1
    EDGE_TYPE = first(edge_types)
    parameters = ["$(n): row.$(n)" for n in filter(x -> !(x in ["TYPE"]), names(table))]
    parameters = "{" * join(parameters, ", ") * "}"

    ProgressMeter.@showprogress for (i, window) in enumerate(windows)
        df_sub = table[window, :]
        f = "edge$i.tsv"
        local_f_path = "$(neo4j_import_dir)/$(f)"
        uCSV.write(local_f_path, df_sub, delim='\t')
        run(`chmod 777 $(local_f_path)`)
        f_url = "file:///$(f)"
        cmd =
        """
        LOAD CSV WITH HEADERS FROM '$(f_url)' AS row FIELDTERMINATOR '\t'
        MATCH (src {identifier: row.src})
        MATCH (dst {identifier: row.dst})
        MERGE (src)-[p:`$(EDGE_TYPE)` $(parameters)]->(dst)
        """
        # had been merge above!
        # create Progress:   2%|▊                                        |  ETA: 1:04:39
        # merge Progress:   3%|█▍                                       |  ETA: 1:25:12
        cmd = rstrip(replace(cmd, '\n' => ' '))
        cypher_cmd = Mycelia.cypher(cmd, address = address, username = username, password = password, database = database)
        run(cypher_cmd)
        # rm(local_f_path)
    end
end

In [ ]:
upload_edges_to_neo4j(graph=graph, address=local_neo4j_bolt_address, password=local_neo4j_password, neo4j_import_directory=neo4j_local_import_directory)

In [ ]:
run(Mycelia.cypher("MATCH (n1)-[r]->(n2) return count(r)", address=local_neo4j_bolt_address, password=local_neo4j_password))

In [ ]:
run(Mycelia.cypher("MATCH (n1)-[r]->(n2) return r LIMIT 25", address=local_neo4j_bolt_address, password=local_neo4j_password))

In [ ]:
Graphs.ne(graph)

In [ ]:
datetime = replace(string(Dates.now()), r"[\-:.a-zA-Z]" => "")
neo4j_dump = "$(working_directory)/neo4j-covid-database-$(n)-$(datetime).dump"

In [ ]:
run(`$(NEO4J_PATH)/bin/neo4j stop`)
run(`$(NEO4J_PATH)/bin/neo4j-admin dump --database=neo4j --to=$(neo4j_dump)`)
run(`$(NEO4J_PATH)/bin/neo4j start`)

In [ ]:
println("scp $(neo4j_dump) cameron@odin.lcfta.com:/mnt/coldstorage/neo4j/import/$(basename(neo4j_dump))")

to reload new database
- `scp local/path/to/db.dump cameron@odin.lcfta.com:/mnt/coldstorage/neo4j/import/db.dump`
- `ssh cameron@odin.lcfta.com`
- `sudo docker stop neo4j && sudo docker rm neo4j`
- `sudo docker stop neo4jloader && sudo docker rm neo4jloader`
- `sudo chmod -R 777 /mnt/coldstorage/neo4j && sudo bash /mnt/coldstorage/git/docker-neo4j/loaddb.sh /var/lib/neo4j/import/neo4j-covid-database-10-20221014181634424.dump`
    - the above now contains `/var/lib/neo4j/bin/neo4j-admin load --from=/var/lib/neo4j/import/neo4j-covid-database-10-20221001235948609.dump --database=neo4j --force"`
- `cd /mnt/coldstorage/git/docker-neo4j && sudo ./reinit.sh`

get container ID
- `sudo docker container ls`